In [1]:
import dash
from dash import dcc, html, dash_table
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output, State

# Initialize sample data (empty expense list)
df = pd.DataFrame(columns=["Category", "Amount", "Description"])

# Initialize app
app = dash.Dash(__name__)
app.title = "Expense Tracker"

# Layout
app.layout = html.Div([
    html.H1("Expense Tracker Dashboard", style={"textAlign": "center"}),

    # Form to add expenses
    html.Div([
        html.Div([
            html.Label("Expense Category:"),
            dcc.Dropdown(
                id="input-category",
                options=[
                    {"label": "Food", "value": "Food"},
                    {"label": "Transport", "value": "Transport"},
                    {"label": "Shopping", "value": "Shopping"},
                    {"label": "Bills", "value": "Bills"},
                    {"label": "Other", "value": "Other"},
                ],
                value="Food",
                clearable=False
            )
        ], style={"marginBottom": "15px"}),

        html.Div([
            html.Label("Amount:"),
            dcc.Input(id="input-amount", type="number", placeholder="Enter amount", value=0, step=1),
        ], style={"marginBottom": "15px"}),

        html.Div([
            html.Label("Description:"),
            dcc.Input(id="input-description", type="text", placeholder="Enter description"),
        ], style={"marginBottom": "15px"}),

        html.Button("Add Expense", id="add-btn", n_clicks=0, style={"marginTop": "10px"})
    ], style={"width": "40%", "margin": "auto", "padding": "20px", "border": "1px solid #ccc", "borderRadius": "10px"}),

    html.Hr(),

    # Expense Table
    html.H2("Expenses List", style={"textAlign": "center"}),
    dash_table.DataTable(
        id="expense-table",
        columns=[
            {"name": "Category", "id": "Category"},
            {"name": "Amount", "id": "Amount"},
            {"name": "Description", "id": "Description"}
        ],
        data=df.to_dict("records"),
        style_table={"width": "70%", "margin": "auto"},
        style_cell={"textAlign": "center"},
    ),

    html.Hr(),

    # Graphs (Bar + Pie side by side)
    html.H2("Expenses by Category", style={"textAlign": "center"}),
    html.Div([
        dcc.Graph(id="expense-bar", style={"display": "inline-block", "width": "48%"}),
        dcc.Graph(id="expense-pie", style={"display": "inline-block", "width": "48%"})
    ], style={"textAlign": "center"})
])

# Store data in memory
expenses_data = df.copy()

# Callback to update table & graphs when adding expense
@app.callback(
    [Output("expense-table", "data"),
     Output("expense-bar", "figure"),
     Output("expense-pie", "figure")],
    [Input("add-btn", "n_clicks")],
    [State("input-category", "value"),
     State("input-amount", "value"),
     State("input-description", "value")]
)
def update_expenses(n_clicks, category, amount, description):
    global expenses_data

    if n_clicks > 0 and amount is not None and amount > 0:
        # Add new expense
        new_row = {"Category": category, "Amount": amount, "Description": description if description else ""}
        expenses_data = pd.concat([expenses_data, pd.DataFrame([new_row])], ignore_index=True)

    # Create summary graphs
    if not expenses_data.empty:
        summary = expenses_data.groupby("Category", as_index=False)["Amount"].sum()

        bar_fig = px.bar(summary, x="Category", y="Amount", title="Total Expenses by Category", text_auto=True)
        pie_fig = px.pie(summary, names="Category", values="Amount", title="Expenses Distribution")
    else:
        bar_fig = px.bar(title="No Expenses Yet")
        pie_fig = px.pie(title="No Expenses Yet")

    return expenses_data.to_dict("records"), bar_fig, pie_fig


# Run server
if __name__ == "__main__":
    app.run(debug=True)
